In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import math
from sklearn.metrics import mean_squared_error

In [11]:
df=pd.read_csv("/content/IBM_2006-01-01_to_2018-01-01.csv",index_col="Date",parse_dates=['Date'])
df.head()

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM


In [12]:
mytrain=df[:'2016'].iloc[:,1:2].values
mytest=df['2017':].iloc[:,1:2].values

In [13]:
print(mytrain[0])
print(mytest[0])

[82.55]
[167.87]


In [14]:
sc=MinMaxScaler(feature_range=(0,1))
mytrain_scaled=sc.fit_transform(mytrain)
mytrain_scaled


array([[0.06065089],
       [0.06029868],
       [0.06311637],
       ...,
       [0.66074951],
       [0.65546633],
       [0.6534235 ]])

In [15]:
len(mytrain_scaled)

2769

In [16]:
I_train=[]
O_train=[]
for i in range(60,2769):#len(mytrain_scaled
  I_train.append(mytrain_scaled[i-60:i,0])
  O_train.append(mytrain_scaled[i,0])
print(I_train[0])
print(O_train[0])

[0.06065089 0.06029868 0.06311637 0.0781206  0.07262609 0.07171034
 0.07657087 0.07058326 0.0669907  0.06494787 0.075796   0.07361229
 0.06417301 0.05621302 0.05783319 0.05409975 0.05431107 0.05515638
 0.05543815 0.05677656 0.05846717 0.05388842 0.04811214 0.04233587
 0.04402649 0.0490279  0.04832347 0.05297267 0.05614258 0.05290223
 0.05325444 0.04909834 0.04994365 0.04797126 0.05431107 0.05212736
 0.04726684 0.04895745 0.04656241 0.04839391 0.04416737 0.0485348
 0.04719639 0.04825303 0.05395886 0.05663567 0.05853762 0.05959425
 0.06375035 0.06917442 0.06889265 0.06670893 0.06910397 0.07783883
 0.07565511 0.07276698 0.06889265 0.0656523  0.06656805 0.06769513]
0.06875176105945335


In [17]:
I_train=np.array(I_train)
O_train=np.array(O_train)
I_train.shape

(2709, 60)

In [18]:
I_train=I_train.reshape(2709,60,1)
print(I_train)

[[[0.06065089]
  [0.06029868]
  [0.06311637]
  ...
  [0.0656523 ]
  [0.06656805]
  [0.06769513]]

 [[0.06029868]
  [0.06311637]
  [0.0781206 ]
  ...
  [0.06656805]
  [0.06769513]
  [0.06875176]]

 [[0.06311637]
  [0.0781206 ]
  [0.07262609]
  ...
  [0.06769513]
  [0.06875176]
  [0.06804734]]

 ...

 [[0.594745  ]
  [0.59587208]
  [0.59094111]
  ...
  [0.66420118]
  [0.65898845]
  [0.66244012]]

 [[0.59587208]
  [0.59094111]
  [0.58812342]
  ...
  [0.65898845]
  [0.66244012]
  [0.66074951]]

 [[0.59094111]
  [0.58812342]
  [0.59002536]
  ...
  [0.66244012]
  [0.66074951]
  [0.65546633]]]


In [19]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(I_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(units=1))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 60, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 60, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 60, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 60, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,051 (277.54 KB)

 Trainable params: 71,051 (277.54 KB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
model.compile(optimizer='rmsprop',loss='mean_squared_error')

In [22]:
model.fit(I_train,O_train,epochs=50,batch_size=32)

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 15s 108ms/step - loss: 0.1290
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - loss: 0.0807
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - loss: 0.0765
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 150ms/step - loss: 0.0773
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 16s 100ms/step - loss: 0.0771
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - loss: 0.0754
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 109ms/step - loss: 0.0735
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 109ms/step - loss: 0.0744
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - loss: 0.0747
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - loss: 0.0734
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 10s 118ms/step - loss: 0.0718
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 109ms/step - loss: 0.0711
Epoch 13/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 111ms/step - loss: 0.0751
Epoch 14/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - loss: 0.0721
Epoch 15/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/s

In [23]:
model.save('IBM_LSTM.h5')

In [24]:
from tensorflow.keras.models import load_model
import numpy as np
model=load_model('IBM_LSTM.h5')


In [25]:
#60 values
user_input = [83.58746017, 80.34122742, 80.81285462, 82.00291895, 83.29159961, 81.44688209, 80.76357678, 83.80165525, 82.51120409, 82.48569342, 80.65383659, 81.69740582, 82.78338862, 83.37704145, 83.53829805, 80.20374715, 80.42476455, 82.69989097, 83.73417143, 82.95666579, 83.94381372, 82.37856671, 83.60787851, 81.80850007, 82.61239819, 82.06021048, 81.0673498, 83.7835985, 81.74964384, 83.10783276, 80.54146548, 80.49215929, 82.03237911, 81.79802128, 80.93914228, 83.90444224, 83.2143073, 81.31844381, 83.23807452, 83.4603476, 81.85039395, 81.3031393, 80.6351211, 83.65161771, 80.54893536, 82.22940581, 82.50289209, 80.59368723, 83.24620825, 81.77694404, 83.20275117, 83.43996357, 83.38335394, 82.69282925, 80.80822433, 80.30044039, 83.6842126, 82.82960781, 80.12425038, 82.25607533]

#2D array
user_input_array = np.array(user_input).reshape(60, 1)

#down-scale
sc = MinMaxScaler(feature_range=(0,1))
user_scaled = sc.fit_transform(user_input_array)

#3D array
user_scaled = user_scaled.reshape(1, 60, 1)

#predict
pred = model.predict(user_scaled)
print(pred) #down scale output
#up-scale
pred_original = sc.inverse_transform(pred.reshape(-1, 1))
print("The stock price is",pred_original[0][0]) #up scaled/original o/p

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
[[[0.5250919 ]
  [0.5367838 ]
  [0.5325145 ]
  [0.52897805]
  [0.52723515]
  [0.5265748 ]
  [0.52639765]
  [0.52668077]
  [0.5272078 ]
  [0.5278347 ]
  [0.5282968 ]
  [0.5286087 ]
  [0.5288989 ]
  [0.52926904]
  [0.5297545 ]
  [0.5300213 ]
  [0.529987  ]
  [0.529896  ]
  [0.52994794]
  [0.5301204 ]
  [0.5304781 ]
  [0.53087294]
  [0.5313529 ]
  [0.53174853]
  [0.5320735 ]
  [0.53228265]
  [0.5322684 ]
  [0.5322843 ]
  [0.53222424]
  [0.53218836]
  [0.531964  ]
  [0.5314949 ]
  [0.530961  ]
  [0.53042316]
  [0.52982575]
  [0.52946186]
  [0.5293543 ]
  [0.5292922 ]
  [0.52938145]
  [0.5296657 ]
  [0.52997154]
  [0.5301659 ]
  [0.5301416 ]
  [0.5301848 ]
  [0.5300827 ]
  [0.52993834]
  [0.5298295 ]
  [0.5295915 ]
  [0.52944887]
  [0.5293362 ]
  [0.5293635 ]
  [0.52957964]
  [0.52996826]
  [0.5304278 ]
  [0.5307167 ]
  [0.5307071 ]
  [0.53072715]
  [0.5308072 ]
  [0.53067327]
  [0.53046906]]]
The stock price is 82.12987
